In [1]:
#librerias
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import json
import pandas as pd
from datetime import datetime
import numpy as np

In [2]:
##Nuevas
def buscar(elemento,busqueda):
    WebDriverWait(driver, 10)\
        .until(EC.element_to_be_clickable((By.CSS_SELECTOR,elemento)))\
        .send_keys(busqueda)
    
def Click(elemento):
    WebDriverWait(driver, 5)\
        .until(EC.element_to_be_clickable((By.CSS_SELECTOR,elemento))).click()

def clear_buscar(elemento):
    WebDriverWait(driver, 20)\
        .until(EC.element_to_be_clickable((By.CSS_SELECTOR,elemento)))\
        .clear()
    
#Se solicita el Xfullpath para hacer click
def Click_Xpath(path):
    WebDriverWait(driver, 15)\
    .until(EC.element_to_be_clickable((By.XPATH,path)))\
    .click()
    
    
##Funcion NoClick hace referencia a los apartados que no son clikeables y se accede a el por medio del FullXpath
def NoClick(path):
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH,path)))
    url=driver.find_element("xpath",path).get_attribute("href")
    driver.get(url)   
    
def ListarNombres(path,clase):
    WebDriverWait(driver, 8).until(EC.presence_of_element_located((By.XPATH,path)))
    html_list=driver.find_element(By.CSS_SELECTOR,clase)
    lista = html_list.find_elements(By.TAG_NAME,'div')
    titulo=[]
    for i in lista:
        text = i.get_attribute('innerHTML')
        soup = BeautifulSoup(text,'html.parser')
        titulo.append(soup.findAll("h2")[0].string)
        #print(titulo)
    return titulo

def ListarPrecios(path,clase):
    WebDriverWait(driver, 8).until(EC.presence_of_element_located((By.XPATH,path)))
    html_list=driver.find_element(By.CSS_SELECTOR,clase)
    lista = html_list.find_elements(By.TAG_NAME,'li')
    
    for i in lista:
        text = i.get_attribute('innerHTML')
        soup = BeautifulSoup(text,'html.parser')
        precio = soup.select("span.price-tag-fraction")[0].text
        print(precio)
def LeerJson(path):
    
    WebDriverWait(driver, 25).until(EC.presence_of_element_located((By.XPATH,path)))
    Json_Jumbo=driver.find_element("xpath",path).get_attribute("text")
    Json_Jumbo = json.loads(Json_Jumbo)
    DatosProducto = Json_Jumbo['itemListElement'] # Se toma el Json anidado que contiene los datos del producto
    return DatosProducto

In [3]:
#Funciones para limpieza y procesamiento de datos 
def cantidad(producto):
    
    l=len(producto)
    producto=producto[::-1] #Invertimos la cadena para encontrar x
    m =producto.find("x")
    producto = producto[:m] #Esta invertimo el valor
    producto=producto[::-1] #Lo volvemos a su estado normal
    return producto


In [5]:
#Ruta de chorme compatible 
driver_path = "C:\\Users\\danie\\OneDrive\\Documentos\\Notebooks\\chromedriver.exe"
## Opciones con las que inicia google chrome 
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument("--headless")
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')

s = Service(driver_path)#Se agrega el path como objeto para evitar errores 


# Main

## Función buscar:
- Recibe como parametro el input class del cuadro de busqueda de la pagina y el producto a buscar
- Se antepone input antes de colocar el input class
- Si el input class tiene espacios estos se remplazan por puntos

## Funcion Click
- Recibe como parametro el elemento clikeable anteponiendo el formato(div, footer, etc)


## Función ListarProductos
- Esta funcion lista todos los productos de la busquema realizada 


In [8]:
driver = webdriver.Chrome(service=s)
driver.maximize_window()
driver.get("https://www.tiendasjumbo.co/") 

ListaProducto=['arroz y granos','aceite','Enlatados Y Conservas','Carnes Frías Y Embutidos']
CantMas = dict([('arroz y granos', 10),('aceite', 3),('Enlatados Y Conservas', 5),('Carnes Frías Y Embutidos', 3)])


Json_Total=[]
time.sleep(2)
for i in range(len(ListaProducto)):
    time.sleep(1)
#Buscar producto
    producto = ListaProducto[i]
    input_class = 'input.vtex-styleguide-9-x-input.ma0.border-box.vtex-styleguide-9-x-hideDecorators.vtex-styleguide-9-x-noAppearance.br2.br-0.br--left.w-100.bn.outline-0.bg-base.c-on-base.b--muted-4.hover-b--muted-3.t-body.pl5'
    buscar(input_class,producto)
    time.sleep(2)
#Click en buscar
    Element_clickeable = 'footer.vtex-search-2-x-tileListFooter'
    Click(Element_clickeable)
    
#Scroll
    Element_clickeable2 = 'a.vtex-button.bw1.ba.fw5.v-mid.relative.pa0.lh-solid.br2.min-h-small.t-action--small.bg-action-primary.b--action-primary.c-on-action-primary.hover-bg-action-primary.hover-b--action-primary.hover-c-on-action-primary.pointer.inline-flex.items-center.no-underline'
    for j in range(CantMas[ListaProducto[i]]):
        Click(Element_clickeable2)
        time.sleep(2)
#Guardar en Json     
    time.sleep(1)
    
    driver.set_page_load_timeout(30)
    path= '/html/body/div[2]/div/div[1]/div/div[4]/div/div/section/div[2]/div/div[4]/div/div[2]/div/div/div[2]/div/div[5]/div/div/div/script'
    Json_Total += LeerJson(path) #Esto deja un JSON con el que se trabajara los datos
#limpiar
    if(i>=1):
        driver.get("https://www.tiendasjumbo.co/") 
#driver.close()
print('Finalizo correctamente el proceso')

Finalizo correctamente el proceso


# Procesamiento de datos

In [9]:
#Se obtienen los datos del segundo nivel de profundidad del json y se pasan a un dataframe
Producto_1 = pd.json_normalize(Json_Total) 
#Se accede al 3 nivel de produndidad del json 
JsonAnidado=[] #Se crea una lista vacia para llenarla con el for

for i in Producto_1['item.offers.offers']:
    JsonAnidado +=i 
    
#Se normaliza el nivel 3 de profundidad del json    
Producto_2= pd.json_normalize(JsonAnidado) # Hacemos join de los 2 dataframe creados
Productos_T = pd.concat([Producto_1, Producto_2], axis = 1) 

In [10]:
#Seleccionamos las columnas que nos interesan
Productos_T = Productos_T[['sku','item.name','item.brand.name','item.description','priceCurrency','price','priceValidUntil','item.@id','item.image','seller.name']]
#Renombramos las columnas 
Productos_T.columns = ['Codigo_Pro','Nombre', 'Marca', 'Descripcion', 'Moneda', 'Precio','Precio_Valido_Hasta','Url_producto', 'Url_Imagen','Almacen']

# Limpieza de datos

In [14]:
Productos_T['Cantidad']=Productos_T['Nombre'].apply(lambda x: cantidad(x))
Productos_T['Fecha']= now.strftime('%Y-%m-%d')
Productos_T['Categoria']=Productos_T['Nombre'].apply(lambda x: Categoria(x))

In [11]:
def Categoria (producto):
    producto = producto.split(' ', 1)
    return producto

## Exporta archivo json


In [13]:
#Obtenemos la fecha actual
now = datetime.now()
format = now.strftime('%Y-%m-%d')
NombreAlmacen='Jumbo-'+format

Productos_T.to_json(f'{NombreAlmacen}.json', orient = 'split', compression = 'infer', index = 'true') 

In [63]:
Productos_T.to_csv('Jumbo.csv', index=False,sep='|',encoding='utf-8')

In [15]:
Productos_T

,Codigo_Pro,Nombre,Marca,Descripcion,Moneda,Precio,Precio_Valido_Hasta,Url_producto,Url_Imagen,Almacen,Cantidad,Fecha,Categoria
0,22018,Arroz vitarroz plus Supremo x 10 kg,SUPREMO,Arroz Supremo Premium es símbolo de la excelen...,COP,36590,2023-08-28T22:06:36Z,https://www.tiendasjumbo.co/arroz-vitarroz-plu...,https://jumbocolombiaio.vtexassets.com/arquivo...,Cencosud - Colombia,10 kg,2022-10-03,"[Arroz, vitarroz plus Supremo x 10 kg]"
1,111865,Arroz Cuisine&amp;Co blanco x10000g,CUISINE &amp; CO NBE MP,Arroz cuisine&amp;co blanco x10000g,COP,37490,2023-08-28T22:30:36Z,https://www.tiendasjumbo.co/arroz-cuisine-co-b...,https://jumbocolombiaio.vtexassets.com/arquivo...,Cencosud - Colombia,10000g,2022-10-03,"[Arroz, Cuisine&amp;Co blanco x10000g]"
2,13397,Arroz Diana x 10 Kg,DIANA,Proporciónale todo el amor a tu familia con el...,COP,36890,2023-09-27T00:00:00Z,https://www.tiendasjumbo.co/arroz-diana-x-10-kg/p,https://jumbocolombiaio.vtexassets.com/arquivo...,Cencosud - Colombia,10 Kg,2022-10-03,"[Arroz, Diana x 10 Kg]"
3,13396,Arroz Diana x 5 Kg,DIANA,Proporciónale todo el amor a tu familia con el...,COP,19490,2023-08-28T22:22:13Z,https://www.tiendasjumbo.co/arroz-diana-x-5-kg/p,https://jumbocolombiaio.vtexassets.com/arquivo...,Cencosud - Colombia,5 Kg,2022-10-03,"[Arroz, Diana x 5 Kg]"
4,22006,Arroz Supremo x 5 kg,SUPREMO,Arroz Supremo contiene una mezcla de vitaminas...,COP,19590,2023-06-12T22:04:08Z,https://www.tiendasjumbo.co/arroz-supremo-x-5-...,https://jumbocolombiaio.vtexassets.com/arquivo...,Cencosud - Colombia,5 kg,2022-10-03,"[Arroz, Supremo x 5 kg]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
391,61619,Salchichón La Fazenda cervecero premium x 500 g,LA FAZENDA,"Día a día, la Fazenda es más querida por los c...",COP,10590,2023-09-30T22:03:37Z,https://www.tiendasjumbo.co/salchichon-la-faze...,https://jumbocolombiaio.vtexassets.com/arquivo...,Cencosud - Colombia,500 g,2022-10-03,"[Salchichón, La Fazenda cervecero premium x 50..."
392,198967,Combo Zenú salchichón de pollo x250g + mortade...,ZENU,Cbozenuschonpllx250g+mtdelpllx500ggrtsalchpllx...,COP,16590,2023-07-19T22:04:58Z,https://www.tiendasjumbo.co/cbozenuschonpllx25...,https://jumbocolombiaio.vtexassets.com/arquivo...,Cencosud - Colombia,250g,2022-10-03,"[Combo, Zenú salchichón de pollo x250g + morta..."
393,57623,Jamón Pietrán cerdo Premium x 450g,PIETRAN,Su exquisita combinación de pierna de cerdo ma...,COP,24590,2023-07-19T22:03:13Z,https://www.tiendasjumbo.co/jamon-pietran-cerd...,https://jumbocolombiaio.vtexassets.com/arquivo...,Cencosud - Colombia,450g,2022-10-03,"[Jamón, Pietrán cerdo Premium x 450g]"
394,11606,Salchichón Cervecero Premium Zenú x 500 g.,ZENU,Salchicha con buena fuente de proteína ideal p...,COP,19990,2023-09-19T22:01:43Z,https://www.tiendasjumbo.co/salchichon-cervece...,https://jumbocolombiaio.vtexassets.com/arquivo...,Cencosud - Colombia,500 g.,2022-10-03,"[Salchichón, Cervecero Premium Zenú x 500 g.]"
